In [1]:
import os
import sys
os.chdir("/Users/ash/Downloads/capstone_code")
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from visualization.plotting import plot_pose, plot_pose_visible
import pandas as pd
import numpy as np
#Load data and plot some examples
data = pd.read_csv('./data/penn/processed_penn_data.csv', index_col=[0]).reset_index(drop=True)

for n in np.random.randint(0, data.shape[0], 3):
    plot_pose_visible(data.iloc[n, 2:15], data.iloc[n, 15:28], list(data.iloc[n, 28:41]), data.iloc[n, -4], data.iloc[n, -3])
    print("ID", data.iloc[n, 0])
    print("frame", data.iloc[n, 1])

[1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]


<Figure size 1200x800 with 1 Axes>

ID 669
frame 9
[1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]


<Figure size 1200x800 with 1 Axes>

ID 1469
frame 2
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


<Figure size 1200x800 with 1 Axes>

ID 1281
frame 68


In [3]:
ID = 1571

df = data[data['file_id'] == ID].sort_values(['frame']).reset_index(drop=True)

In [4]:
def augment_position(xs, ys, n_aug):
    max_x = np.max(np.array(xs).reshape(-1))
    min_x = np.min(np.array(xs).reshape(-1))
    max_y = np.max(np.array(ys).reshape(-1))
    min_y = np.min(np.array(ys).reshape(-1))
    x_aug = [-i*(min_x - 0)/n_aug for i in range(0, n_aug+1)] + [i*(1-max_x)/n_aug for i in range(0, n_aug+1)]
    y_aug = [-i*(min_y - 0)/n_aug for i in range(0, n_aug+1)] + [i*(1-max_y)/n_aug for i in range(0, n_aug+1)]
    augmented_x = []
    augmented_y = []
    for move_x in x_aug: 
        for move_y in y_aug:
            augmented_x.append(np.array(xs)+move_x*np.ones(shape=(xs.shape[0], xs.shape[1])))
            augmented_y.append(np.array(ys)+move_y*np.ones(shape=(ys.shape[0], ys.shape[1])))
    return np.array(augmented_x), np.array(augmented_y)

def augment_size(xs, ys):
    augmented_x, augmented_y = [], []
    for shrink in np.linspace(0.1, 1.0, 10):
        augmented_x.append(xs*shrink)
        augmented_y.append(ys*shrink)
    return np.array(augmented_x), np.array(augmented_y)

In [5]:
def serialize_data_clf(nframe, data, n_aug, save=False, save_path=None):
    import joblib
    train_input = -np.ones(shape=(1, nframe, 13*3))
    train_label = []
    for ID in data['file_id'].unique():
        df = data[data['file_id'] == ID].sort_values(['frame']).reset_index(drop=True)

        df.iloc[:, 2:15] = df.iloc[:, 2:15]/df['w'].iloc[0]
        df.iloc[:, 15:28] = df.iloc[:, 15:28]/df['h'].iloc[0]
        if n_aug > 0:
            xs, ys = augment_position(df.iloc[:, 2:15], df.iloc[:, 15:28], n_aug)

            _class = df['action']
            X_train = []
            y_train = []
            for n in range(xs.shape[0]):
                for i in range(nframe, df.shape[0]):
                    curr = np.append(xs[n,i-nframe:i,:], ys[n,i-nframe:i,:], axis=1)
                    curr = np.append(curr, df.iloc[i-nframe:i, 28:41], axis=1)
                    X_train.append(curr)
                    y_train.append(_class[i])
        else: 
            _class = df['action']
            X_train = []
            y_train = []
            for i in range(nframe, df.shape[0]):
                X_train.append(np.array(df.iloc[i-nframe:i, 2:41]))
                y_train.append(_class[i])
        X_train = np.array(X_train)

        train_input = np.append(train_input, X_train, axis=0)
        train_label = train_label + y_train
    train_input = train_input[1:, :, :]
    train_label = np.array(train_label)

    from sklearn.preprocessing import OneHotEncoder

    encoder = OneHotEncoder()
    train_label = encoder.fit_transform(train_label.reshape(-1,1)).toarray()
    
    if save:
        train_input.dump(save_path+"clf_input_{}-nframe_{}-augmented".format(nframe, n_aug))
        train_label.dump(save_path+"clf_label_{}-nframe_{}-augmented".format(nframe, n_aug))
        joblib.dump(encoder, save_path+"clf_encoder_{}-nframe_{}-augmented".format(nframe, n_aug))
    
    return train_input, train_label, encoder

In [5]:
def serialize_data_next_frame(nframe, data, n_aug, save=False, save_path=None):
    train_input = -np.ones(shape=(1, nframe, 13*3))
    train_label = -np.ones(shape=(1, 13*3))
    for ID in data['file_id'].unique():
        df = data[data['file_id'] == ID].sort_values(['frame']).reset_index(drop=True)

        df.iloc[:, 2:15] = df.iloc[:, 2:15]/df['w'].iloc[0]
        df.iloc[:, 15:28] = df.iloc[:, 15:28]/df['h'].iloc[0]
        if n_aug > 0:
            xs, ys = augment_position(df.iloc[:, 2:15], df.iloc[:, 15:28], n_aug)

            _class = df['action']
            X_train = []
            y_train = []
            for n in range(xs.shape[0]):
                for i in range(nframe, df.shape[0]):
                    curr = np.append(xs[n,i-nframe:i+1,:], ys[n,i-nframe:i+1,:], axis=1)
                    curr = np.append(curr, df.iloc[i-nframe:i+1, 28:41], axis=1)
                    X_train.append(curr[:-1])
                    y_train.append(curr[-1])
        else: 
            X_train = []
            y_train = []
            for i in range(nframe, df.shape[0]):
                X_train.append(np.array(df.iloc[i-nframe:i, 2:41]))
                y_train.append(np.array(df.iloc[i, 2:41]))
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        train_input = np.append(train_input, X_train, axis=0)
        train_label = np.append(train_label, y_train, axis=0)
    train_input = train_input[1:, :, :]
    train_label = train_label[1:,:]

    if save:
        train_input.dump(save_path+"nf_input_{}-nframe_{}-augmented".format(nframe, n_aug))
        train_label.dump(save_path+"nf_label_{}-nframe_{}-augmented".format(nframe, n_aug))
        
    return train_input, train_label

In [6]:
augmented = serialize_data_next_frame(5, data.iloc[:1000], 0)

In [7]:
a, b = augmented

In [52]:
def preprocess_input_clf(_input):
    #pertube input
    n_aug = 10
    xs = _input[:,:13]
    ys = _input[:,13:26]
    max_x = np.max(np.array(xs).reshape(-1))
    min_x = np.min(np.array(xs).reshape(-1))
    max_y = np.max(np.array(ys).reshape(-1))
    min_y = np.min(np.array(ys).reshape(-1))
    x_aug = [-i*(min_x - 0)/n_aug for i in range(0, n_aug+1)] + [i*(1-max_x)/n_aug for i in range(0, n_aug+1)]
    y_aug = [-i*(min_y - 0)/n_aug for i in range(0, n_aug+1)] + [i*(1-max_y)/n_aug for i in range(0, n_aug+1)]
    move_x = np.random.choice(x_aug)
    move_y = np.random.choice(y_aug)
    new_xs = np.array(xs)+move_x*np.ones(shape=(xs.shape[0], xs.shape[1]))
    new_ys = np.array(ys)+move_y*np.ones(shape=(ys.shape[0], ys.shape[1]))
    result = np.append(new_xs, new_ys, axis=1)
    result = np.append(result, _input[:,-13:], axis=1)
    return result

def get_input_clf(_id, train_input):
    result = preprocess_input_clf(train_input[_id])
    return result


def get_output_clf(_id, train_label):
    return train_label[_id]


def data_generator_clf(train_input, train_label, batch_size = 64):   
    while True:
        batch_idx = np.random.choice(range(0, train_input.shape[0]), 
                                     size = batch_size)
        shape = (1, train_input.shape[1], 13*3)
        batch_input = np.zeros(shape=shape)
        batch_output = np.zeros(shape=(1, train_label.shape[1])) 
        for idx in batch_idx:
            _input = get_input_clf(idx, train_input).reshape(1, batch_input.shape[1], batch_input.shape[2])
            _output = get_output_clf(idx, train_label).reshape(1, batch_output.shape[1])
            batch_input = np.append(batch_input, _input, axis=0)
            batch_output = np.append(batch_output, _output, axis=0)
        batch_x = batch_input[1:]
        batch_y = batch_output[1:]
        
        yield batch_x, batch_y

In [8]:
def preprocess_input_nf(_input):
    #pertube input
    n_aug = 10
    xs = _input[:,:13]
    ys = _input[:,13:26]
    max_x = np.max(np.array(xs).reshape(-1))
    min_x = np.min(np.array(xs).reshape(-1))
    max_y = np.max(np.array(ys).reshape(-1))
    min_y = np.min(np.array(ys).reshape(-1))
    x_aug = [-i*(min_x - 0)/n_aug for i in range(0, n_aug+1)] + [i*(1-max_x)/n_aug for i in range(0, n_aug+1)]
    y_aug = [-i*(min_y - 0)/n_aug for i in range(0, n_aug+1)] + [i*(1-max_y)/n_aug for i in range(0, n_aug+1)]
    move_x = np.random.choice(x_aug)
    move_y = np.random.choice(y_aug)
    new_xs = np.array(xs)+move_x*np.ones(shape=(xs.shape[0], xs.shape[1]))
    new_ys = np.array(ys)+move_y*np.ones(shape=(ys.shape[0], ys.shape[1]))
    result = np.append(new_xs, new_ys, axis=1)
    result = np.append(result, _input[:,-13:], axis=1)
    return result, move_x, move_y

def get_input_nf(_id, train_input):
    result, move_x, move_y = preprocess_input_nf(train_input[_id])
    return result, move_x, move_y


def get_output_nf(_id, train_label, move_x, move_y):
    xs = train_label[_id][:13]
    ys = train_label[_id][13:26]
    new_xs = np.array(xs)+move_x*np.ones(shape=(xs.shape[0]))
    new_ys = np.array(ys)+move_y*np.ones(shape=(ys.shape[0]))
    result = np.append(new_xs.reshape(1, new_xs.shape[0]), new_ys.reshape(1, new_ys.shape[0]), axis=1)
    result = np.append(result, train_label[_id,-13:].reshape(1,13), axis=1)
    return result


def data_generator_nf(train_input, train_label, batch_size = 64, augmented=True):   
    while True:
        batch_idx = np.random.choice(range(0, train_input.shape[0]), 
                                     size = batch_size)
        shape = (1, train_input.shape[1], 13*3)
        batch_input = np.zeros(shape=shape)
        batch_output = np.zeros(shape=(1, train_label.shape[1])) 
        if augmented:
            for idx in batch_idx:
                _input, move_x, move_y = get_input_nf(idx, train_input)
                _input = _input.reshape(1, batch_input.shape[1], batch_input.shape[2])
                _output = get_output_nf(idx, train_label, move_x, move_y).reshape(1, 39)
                batch_input = np.append(batch_input, _input, axis=0)
                batch_output = np.append(batch_output, _output, axis=0)
        else:
            for idx in batch_idx:
                _input = train_input[idx].reshape(1, batch_input.shape[1], batch_input.shape[2])
                _output = train_label[idx].reshape(1, 39)
                batch_input = np.append(batch_input, _input, axis=0)
                batch_output = np.append(batch_output, _output, axis=0)
        batch_x = batch_input[1:]
        batch_y = batch_output[1:]
        
        yield batch_x, batch_y

In [9]:
from sklearn.model_selection import train_test_split
train_input, val_input, train_label, val_label = train_test_split(a, b, test_size=0.2)

In [10]:
train_generator = data_generator_nf(train_input, train_label, batch_size=128, augmented=False)

In [13]:
next(train_generator)[1].shape

(128, 39)